In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
import random

In [ ]:
class AgentState(TypedDict):
    player_name: str
    message: str
    guesses: List[int]
    attempts: int
    lower_bound: int
    upper_bound: int
    target_number: int


In [ ]:
def setup(state: AgentState) -> AgentState:
    """Initializes the game state."""

    state["message"] = f"Hi {state['player_name']}, welcome to the Higher-Lower Game!"
    state["guesses"] = []
    state["attempts"] = 0
    state["lower_bound"] = 1
    state["upper_bound"] = 20
    state["target_number"] = random.randint(state["lower_bound"], state["upper_bound"])
    print(state["message"])
    print(f"Target: {state['target_number']}")

    return state

def guess(state: AgentState) -> AgentState:
    """Guesses a number based on the current bounds."""
    
    state["attempts"] += 1
    print(f"\n### Attempt: {state['attempts']}.")
    guess = random.randint(state["lower_bound"], state["upper_bound"])
    state["guesses"].append(guess)
    print(f"Guessing a number: {guess}.")

    print(f"Current bounds: {state['lower_bound']} - {state['upper_bound']}.")

    return state

def hint(state: AgentState) -> AgentState:
    """Provides feedback on the last guess and updates the bounds accordingly."""

    last_guess = state["guesses"][-1]
    target_number = state["target_number"]

    if last_guess < target_number:
        state["lower_bound"] = last_guess + 1
        state["message"] = f"{last_guess} is too low!"

    elif last_guess > target_number:
        state["upper_bound"] = last_guess - 1
        state["message"] = f"{last_guess} is too high!"

    else:
        state["message"] = f"Congratulations {state['player_name']}! You've guessed the number in {state['attempts']} attempts!"

    print(state["message"])
        
    return state
    
def should_continue(state: AgentState) -> AgentState:
    """Determines whether the game should continue or end."""

    if state["target_number"] == state["guesses"][-1]:
        print(f"Congrats {state['player_name']}!")
        return "found"
    
    elif state["attempts"] >= 7:
        print(f"Sorry {state['player_name']}, you've used all your attempts!")
        return "end"
    
    else:
        print(f"Keep going {state['player_name']}! Remaining attempts: {7 - state['attempts']}.")
        return "continue"
    


In [ ]:
graph = StateGraph(AgentState)

graph.add_node("setup_node", setup)
graph.add_node("guess_node", guess)
graph.add_node("hint_node", hint)

graph.add_edge(START, "setup_node")
graph.add_edge("setup_node", "guess_node")
graph.add_edge("guess_node", "hint_node")

graph.add_conditional_edges(
    "hint_node",
    should_continue,
    {
        "continue": "guess_node",
        "found": END,
        "end": END
    }
)

app = graph.compile()

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
app.invoke({"player_name": "Hagi", "guesses": [], "attempts": 0, "lower_bound": 1, "upper_bound": 20})